# SCRAPS of metric testing / creation

### Scraps from writing canopyLayer function

In [ ]:
# Get dict of heights of points for testing
h = lc.height_dict[testkey]

# Get height bins used in processing
hbins = lc.cover_dict[testkey]['HeightBins']

# sort them by height
h = np.sort(h)

# Group each point by height bin
hgroups = np.digitize(h, bins=hbins)

# Count the number of points in each bin
# Note: minlength allows it to go to the full length of hbins, rather than stop at the max height of the points
hcounts = np.bincount(hgroups, minlength=len(hbins))

# Normalize the counts
hcounts_norm = hcounts/np.sum(hcounts)

In [ ]:
# If you want to:
# Interpolate normalized counts to 5 cm increments so that you have more precise height measurements
hbins = np.arange(0, np.max(hbins), 0.05)
hcounts_norm = np.interp(x=hbins, xp=lc.cover_dict[testkey]['HeightBins'], fp=hcounts_norm)

In [ ]:
from scipy.ndimage import gaussian_filter1d
# Smooth point height distribution, and find peaks and troughs\
# Following the below article on stack exchange
# https://stackoverflow.com/questions/62537703/how-to-find-inflection-point-in-python

# smooth with gaussian filter (sigma = 2 m)
smooth = gaussian_filter1d(hcounts_norm, 2)

# compute first derivative
smooth_d1 = np.gradient(smooth)

# find inflection points

# when it switches from positive to negative (a trough)
idx_troughs = np.where(np.diff(np.sign(smooth_d1))>0)[0]

# when it switches from negative to positive (a peak)
idx_peaks = np.where(np.diff(np.sign(smooth_d1))<0)[0]

# output height values of peaks and troughs
troughs = hbins[idx_troughs]
peaks = hbins[idx_peaks]

In [ ]:
fig, ax = plt.subplots()

ax.plot(hcounts_norm, hbins, label='Frequency', lw=2, alpha=0.6)

ax.plot(smooth, hbins, label='Smoothed', lw=2)

for infl in troughs:
    tline = ax.axhline(y=infl, color='b', label='Trough', alpha=0.6)
    
for infl in peaks:
    pline = ax.axhline(y=infl, color='c', label='Peak', alpha=0.6)
    
ax.legend(handles=[tline, pline], loc='best')

ax.set_xlabel('Normalized Frequency of Points')
ax.set_ylabel('Height [m]')
ax.set_xlim(0, np.max(hcounts_norm) + 0.03)

In [ ]:
# Define the gap size as the maximum distance between peaks
gapsize = np.max(np.diff(peaks))

# Number of layers as number of peaks
nlayers = len(peaks)

In [1]:
from scipy.interpolate import UnivariateSpline, CubicSpline, splrep
hbins = lc.cover_dict[(375817.4, 7299326.95)]['HeightBins']
tc = lc.cover_dict[(375817.4, 7299326.95)]

tx_spline = UnivariateSpline(hbins, np.abs(np.gradient(tc['CoverD2'])), k=5)
tx_cubic = CubicSpline(hbins, np.abs(np.gradient(tc['CoverD2'])))
tx_cubic_1d = tx_cubic.derivative()
tx_cubic_2d = tx_cubic_1d.derivative()

hbins_interp = np.arange(0, np.max(hbins), 0.2)
tx_interp = np.interp(x=hbins_interp, xp=hbins[1:], fp=tc['CoverD2byH'])
tx_1d = np.gradient(tx_interp)
tx_2d = np.gradient(tx_1d)
infls = np.where(np.diff(np.sign(tx_1d)))[0]

NameError: name 'lc' is not defined

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.abs(np.gradient(tc['CoverD2'], edge_order=2)), hbins)
ax.plot(np.abs(np.gradient(np.gradient(tc['CoverD2'], edge_order=2))), hbins)

In [ ]:
fig, ax = plt.subplots()
ax.plot(tx_interp, hbins_interp)
for i, infl in enumerate(hbins_interp[infls], 1):
    ax.axhline(y=infl, color='k', label=f'Inflection Point {i}')

In [ ]:
pai_interp = np.abs(np.gradient(tx_interp)) / np.gradient(hbins_interp)
fig, ax = plt.subplots()
ax.plot(pai_interp, hbins_interp)

In [ ]:
tx_cubic_1d.roots()

In [ ]:
fig, ax = plt.subplots()
ax.plot(tx_interp, hbins_interp, 'k', lw=3)
ax.plot(tx_cubic(hbins_interp),hbins_interp, 'g', lw=3)
for infl in tx_cubic_1d.roots():
    ax.axhline(infl)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.abs(np.diff(tc['Npulses'])), hbins[1:])